In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
path = 'Constraint_Train.csv'

In [3]:
df = pd.read_csv(path)
df.head(3)

,id,tweet,label
0,1,The CDC currently reports 99031 deaths. In gen...,real
1,2,States reported 1121 deaths a small rise from ...,real
2,3,Politically Correct Woman (Almost) Uses Pandem...,fake


In [4]:
len(df)

6420

In [5]:
df.label.value_counts()

,count
label,
real,3360
fake,3060


Окей, классы примерно сбалансированы

In [6]:
from nltk.tokenize import TweetTokenizer
import nltk
tk = TweetTokenizer(preserve_case=False)

In [7]:
tweets = [tk.tokenize(tweet) for tweet in df.tweet]

# TopicModelling + LogisticRegression

## Make a topic modelling

In [8]:
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim import similarities

In [9]:
dictionary = Dictionary(tweets)
gensim_corpus = [dictionary.doc2bow(tweet) for tweet in tweets]

In [10]:
tf_idf = TfidfModel(corpus=gensim_corpus)
tf_idf_corpus = tf_idf[gensim_corpus]

In [11]:
from gensim.models import lsimodel

In [12]:
lsi_matrix = lsimodel.LsiModel(corpus=tf_idf_corpus, id2word=dictionary, num_topics=300)

topics = np.array([lsi_matrix[i] for i in gensim_corpus])

## Prepare data

In [13]:
X = topics[:, :, 1]
y = (df.label == 'real').astype('int')

## Work with model

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=12345)

In [16]:
lr = LogisticRegression(random_state=12345)
lr.fit(X_train, y_train)

yhat = lr.predict(X_test)

In [17]:
print(classification_report(y_test, yhat))

              precision    recall  f1-score   support

           0       0.92      0.93      0.93       895
           1       0.94      0.93      0.93      1031

    accuracy                           0.93      1926
   macro avg       0.93      0.93      0.93      1926
weighted avg       0.93      0.93      0.93      1926



# w2v + CNN

## Get text embeddings

In [18]:
from gensim.models import Word2Vec

In [19]:
%%time
word_vect_size = 300
tweet_model = Word2Vec(tweets, vector_size=word_vect_size, min_count=5, window=5, epochs=15)

CPU times: user 7.57 s, sys: 5.65 s, total: 13.2 s
Wall time: 2.46 s


In [20]:
tweet_model.wv.most_similar('wuhan')

[('china', 0.878233015537262),
 ('virology', 0.8698071837425232),
 ('chinese', 0.8117592334747314),
 ('military', 0.8057734966278076),
 ('originated', 0.7962672114372253),
 ('created', 0.7806059718132019),
 ('dead', 0.7690945267677307),
 ('são', 0.7648612260818481),
 ('institute', 0.7645329833030701),
 ('paulo', 0.7627395987510681)]

In [21]:
from collections import Counter

In [22]:
lens = Counter(len(tweet) for tweet in tweets)

In [23]:
def get_embeddings(tokens, max_len=100):
    result = []
    for i in range(max_len):
        if i < len(tokens):
            word = tokens[i]
            if word in tweet_model.wv:
                result.append(tweet_model.wv[word])
            else:
                result.append(np.zeros(word_vect_size))
        else:
            result.append(np.zeros(word_vect_size))
    return result

In [24]:
tweet_embeddings = np.array([get_embeddings(tweet) for tweet in tqdm(tweets)])

100%|██████████| 6420/6420 [00:01<00:00, 5270.97it/s]


In [25]:
tweet_embeddings.shape

(6420, 100, 300)

## Prepare data

In [26]:
import torch

In [27]:
X_train, X_test, y_train, y_test = train_test_split(tweet_embeddings, y, test_size=.3, random_state=12345)

In [28]:
in_data = torch.tensor(X_train).float()
targets = torch.tensor(y_train.values).float()

in_data_test = torch.tensor(X_test).float()
targets_test = torch.tensor(y_test.values).float()

## Build a model

In [29]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [30]:
class my_CNN(nn.Module):
    def __init__(self, word_vect_size, batch_size):
        super(my_CNN, self).__init__()
        self.batch_size = batch_size
        self.conv2 = nn.Conv2d(in_channels=1, out_channels=2, kernel_size=(2, word_vect_size))
        self.conv3 = nn.Conv2d(in_channels=1, out_channels=2, kernel_size=(3, word_vect_size))
        self.conv4 = nn.Conv2d(in_channels=1, out_channels=2, kernel_size=(4, word_vect_size))
        self.drop = nn.Dropout(.3)
        self.softmax = nn.Softmax(-1)
        self.out = nn.Linear(6, 1)
        # self.sigm = nn.Sigmoid()

    def pooling(self, x):
        pool = nn.MaxPool2d((x.shape[2], 1))(x).reshape((x.shape[0], 2))
        # res = torch.cat([pool[0], pool[1]])
        return pool

    def forward(self, x):
        # self.x = x
        x = x.reshape((x.shape[0], 1, x.shape[1], word_vect_size))
        x1 = self.pooling(F.relu(self.conv2(x)))
        x2 = self.pooling(F.relu(self.conv3(x)))
        x3 = self.pooling(F.relu(self.conv4(x)))
        concat_ = torch.cat((x1, x2, x3), dim=1)
        solved = self.out(self.softmax(self.drop(concat_)))

        return solved

net = my_CNN(300, 16)

In [31]:
print(net)

my_CNN(
  (conv2): Conv2d(1, 2, kernel_size=(2, 300), stride=(1, 1))
  (conv3): Conv2d(1, 2, kernel_size=(3, 300), stride=(1, 1))
  (conv4): Conv2d(1, 2, kernel_size=(4, 300), stride=(1, 1))
  (drop): Dropout(p=0.3, inplace=False)
  (softmax): Softmax(dim=-1)
  (out): Linear(in_features=6, out_features=1, bias=True)
)


## Define learning algorythm

In [32]:
optimizer = optim.NAdam(net.parameters(), lr=0.01)
criterion = nn.BCEWithLogitsLoss()

In [33]:
def train_epoch(model, X_train, y_train, batch_size=16, criterion=None, optimizer=None):
    for i in tqdm(range(0, in_data.shape[0], batch_size)):
        batch_x = X_train[i:i+batch_size]
        batch_y = y_train[i:i+batch_size]
        optimizer.zero_grad()
        output = model(batch_x)
        loss = criterion(output.reshape(-1), batch_y)
        loss.backward()
        optimizer.step()
    return loss

def train_model(model, in_data, targets, n_epochs=40, criterion=criterion, optimizer=optimizer):
    for ep in range(1, n_epochs+1):
        print(f'Training epoch {ep}')
        loss = train_epoch(model, in_data, targets, criterion=criterion, optimizer=optimizer)
        print('Loss function result:', loss.data.tolist())

def eval_model(model, in_data_test, targets_test):
    with torch.no_grad():
        output = model(in_data_test).squeeze(1)

    result = np.array((output > 0.5) == targets_test).astype(int)
    print(classification_report(targets_test, result))

## Train and eval model

In [34]:
train_model(net, in_data, targets)

Training epoch 1


100%|██████████| 281/281 [00:02<00:00, 134.82it/s]


Loss function result: 0.45328661799430847
Training epoch 2


100%|██████████| 281/281 [00:02<00:00, 123.45it/s]


Loss function result: 0.39669105410575867
Training epoch 3


...

Loss function result: 0.19545602798461914
Training epoch 39


100%|██████████| 281/281 [00:02<00:00, 137.10it/s]


Loss function result: 0.32014307379722595
Training epoch 40


100%|██████████| 281/281 [00:02<00:00, 136.40it/s]

Loss function result: 0.20660655200481415


In [35]:
eval_model(net, in_data_test, targets_test)

              precision    recall  f1-score   support

         0.0       0.42      0.13      0.19       895
         1.0       0.53      0.85      0.65      1031

    accuracy                           0.51      1926
   macro avg       0.48      0.49      0.42      1926
weighted avg       0.48      0.51      0.44      1926



# w2v + 2way LSTM

Lets leave out our last embeddings

In [36]:
X = tweet_embeddings
y = np.array((df.label == 'real')).astype(int)

In [37]:
X.shape[0] == y.shape[0]

True

## Prepare data

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=15)

In [39]:
in_data = torch.tensor(X_train).float()
targets = torch.tensor(y_train).float()

in_data_test = torch.tensor(X_test).float()
targets_test = torch.tensor(y_test).float()

## Build model

In [40]:
class seryozny_RNN(nn.Module):
    def __init__(self):
        super(seryozny_RNN, self).__init__()
        # self.lstm1 = nn.LSTM(300, 150, 2, dropout=.25)
        self.lstm2 = nn.LSTM(300, 150, bidirectional=True)

        self.pool = nn.MaxPool2d(kernel_size=(1, 300))

        self.dense1 = nn.Linear(100, 1)
        # self.dense2 = nn.Linear(50, 25)
        # self.dense3 = nn.Linear(25, 1)
        self.drop = nn.Dropout(.5)
        self.sigm = nn.Sigmoid()
        # self.res_solver = nn.Linear(100, 1)

    def forward(self, x):
        h0 = torch.randn(2, 100, 150)
        c0 = torch.randn(2, 100, 150)

        # embed, (h1, c1) = self.lstm1(x, (h0, c0))
        embed1, (h2, c2) = self.lstm2(x, (h0, c0))

        x_out = self.pool(embed1).reshape((x.shape[0], 100))

        x_out = self.dense1(x_out)
        # x_out = self.sigm(self.res_solver(x_out.reshape(16, 100)))

        return self.sigm(x_out)
        #

net2 = seryozny_RNN()

## Learning algorythm

In [41]:
optimizer1 = optim.Adam(net2.parameters(), lr=0.01)
criterion1 = nn.BCEWithLogitsLoss()

In [44]:
train_model(net2, in_data, targets, n_epochs=100, criterion=criterion1, optimizer=optimizer1)

Training epoch 1


100%|██████████| 281/281 [00:09<00:00, 29.05it/s]


Loss function result: 0.5314426422119141
Training epoch 2


100%|██████████| 281/281 [00:09<00:00, 29.76it/s]


Loss function result: 0.5302770733833313
Training epoch 3


...

Loss function result: 0.5035220384597778
Training epoch 99


100%|██████████| 281/281 [00:11<00:00, 25.44it/s]


Loss function result: 0.5042172074317932
Training epoch 100


100%|██████████| 281/281 [00:11<00:00, 24.91it/s]

Loss function result: 0.5256994366645813


# Results

In [46]:
print(classification_report(y_test, yhat))

              precision    recall  f1-score   support

           0       0.49      0.49      0.49       910
           1       0.54      0.54      0.54      1016

    accuracy                           0.52      1926
   macro avg       0.52      0.52      0.52      1926
weighted avg       0.52      0.52      0.52      1926



In [47]:
eval_model(net, in_data_test, targets_test)

              precision    recall  f1-score   support

         0.0       0.49      0.13      0.20       910
         1.0       0.53      0.88      0.66      1016

    accuracy                           0.52      1926
   macro avg       0.51      0.50      0.43      1926
weighted avg       0.51      0.52      0.44      1926



In [48]:
eval_model(net2, in_data_test, targets_test)

              precision    recall  f1-score   support

         0.0       0.30      0.07      0.12       910
         1.0       0.51      0.85      0.63      1016

    accuracy                           0.48      1926
   macro avg       0.40      0.46      0.38      1926
weighted avg       0.41      0.48      0.39      1926

